# Entry Queries
Generate queries for different databases for data entry operation

### Libraries

In [2]:
import pandas as pd
import mysql.connector
import os
import pymongo
import redis
from redisgraph import Node, Edge, Graph
import datetime

In [ ]:
%load_ext dotenv
%dotenv

## Loading Data

In [5]:
def load(folder):
    users =  pd.read_csv(f'output/{folder}/users.csv', sep='\t')
    comments = pd.read_csv(f'output/{folder}/comments.csv', sep='\t')
    return users, comments

In [115]:
users, comments = load(input())

### MySQL

In [ ]:
def sql_insert(query, items, conn):
    GROUPS = 10
    total = items.shape[0]
    group = total // GROUPS
    items = list(items.itertuples(name=None, index=None))
    for i in range(GROUPS):
        cursor.executemany(query, items[group * i : group * (i + 1)])
        conn.commit()
        print('#', end='')
    print()

In [ ]:
try:
    conn = mysql.connector.connect(
        host='localhost',
        database='social',
        user=os.getenv('MYSQL_USER'),
        password=os.getenv('MYSQL_PASS'),
        auth_plugin='mysql_native_password'
    )
    cursor = conn.cursor()
    # Deletion
    print("Deleting old records")
    cursor.execute("DELETE FROM comment;")
    cursor.execute("DELETE FROM user;")
    conn.commit()
    # Insertion
    # user
    print("Inserting users")
    query = "INSERT INTO user VALUES(%s, %s, %s, %s)"
    sql_insert(query, users, conn)
    # comment
    print("Inserting comments")
    query = "INSERT INTO comment VALUES(%s, %s, %s, %s)"
    sql_insert(query, comments, conn)
    conn.close()
except mysql.connector.Error as error:
    print(f"Failed to connect to MySQL: {error}")

### MongoDB

In [ ]:
def mongo_insert(col, items):
    GROUPS = 10
    total = len(items)
    group = total // GROUPS
    for i in range(GROUPS):
        col.insert_many(list(items[group * i : group * (i + 1)].T.to_dict().values()))
        print('#', end='')
    print()

In [ ]:
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['test']

user_col = db['user']
comment_col = db['comment']

users = users.rename(columns={'user_id': '_id'})
comments = comments.rename(columns={'comment_id': '_id'})

print("Deleting old records")
user_col.delete_many({})
comment_col.delete_many({})

print("Inserting users")
mongo_insert(user_col, users)
print("Inserting comments")
mongo_insert(comment_col, comments)

users = users.rename(columns={'_id': 'user_id'})
comments = comments.rename(columns={'_id': 'comment_id'})

### Redis

In [118]:
def redis_insert(label, items, graph):
    GROUPS = 10
    total = len(items)
    group = total // GROUPS
    for i in range(GROUPS):
        for item in list(items[group * i : group * (i + 1)].T.to_dict().values()):
            graph.add_node(Node(label=label, properties=item))
        print('#', end='')
    print()
    query = f"""
        MATCH (t:{label})
        WITH t.{label}_id as id, collect(t) AS nodes 
        WHERE size(nodes) >  1
        UNWIND nodes[1..] AS node
        DELETE node
    """
    result = graph.query(query)
    result.pretty_print()
    print()

def redis_rel(graph, comments):
    GROUPS = 10
    total = len(comments)
    group = total // GROUPS
    for i in range(GROUPS):
        for comment in list(comments[group * i : group * (i + 1)].T.to_dict().values()):
            params = {
                'u_id': comment['user_id'],
                'c_id': comment['comment_id']
            }
            query = """
                MATCH (u:user {user_id:$u_id})
                MATCH (c:comment {comment_id:$c_id})
                CREATE (u)-[:wrote]->(c)
            """
            graph.query(query, params)
        print('#', end='')
    print()
        

In [119]:
SOCIAL = 'social'

client = redis.Redis(
    host='localhost',
    port=6379,
)
graph = Graph(SOCIAL, client)

try:
    graph.delete()
except:
    pass

print("Inserting users")
redis_insert('user', users, graph)
print("Inserting comments")
redis_insert('comment', comments, graph)

graph.commit()

print("Creating relations")
redis_rel(graph, comments)

Inserting users
##########
Cached execution 0.0
internal execution time 167.11025

Inserting comments
##########
Cached execution 0.0
internal execution time 0.363408

Creating relations
##########


In [121]:
client = redis.Redis(
    host='localhost',
    port=6379,
)
graph = Graph('social', client)

query = """
    MATCH (u:user)
    RETURN u
"""
# query = """
#     MATCH (u:user)
#     RETURN u
# """
result = len(graph.query(query).result_set)
print(result)

# result = graph.query(query)
# result.pretty_print()

1000
